<a href="https://colab.research.google.com/github/Aniket10-IN/rag_playground/blob/main/Copy_of_rag_philosophy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain \
#   datasets \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \



In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
data[0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator']}

In [ ]:
# pip install --upgrade openai

In [ ]:
import os

In [ ]:
from google.colab import userdata

os.environ['OPENAI_KEY_API'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
data[0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator']}

In [ ]:
import logging
import sys
import os
os.environ[
    "PINECONE_API_KEY"
] = ""
os.environ["OPENAI_API_KEY"] = ""

api_key = os.environ["PINECONE_API_KEY"]

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["quote"], metadata={"source": doc["tags"]}) for doc in (data)
]

In [ ]:
embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)


MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # The maximum number of characters in a chunk: we selected this value arbitrarily
    chunk_overlap=100,  # The number of characters to overlap between chunks
    add_start_index=True,  # If `True`, includes chunk's start index in metadata
    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in RAW_KNOWLEDGE_BASE:
    docs_processed += text_splitter.split_documents([doc])

In [ ]:
# docs_processed

In [ ]:
docs_processed[0]

Document(metadata={'source': ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator'], 'start_index': 0}, page_content='“Be yourself; everyone else is already taken.”')

In [ ]:
index_name = 'pineconebot'

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    docs_processed,
    embeddings,
    index_name = index_name
)

In [ ]:
query = "what does an ideal soulmate do?"

similar_docs = vectorstore.similarity_search(query)

In [ ]:
similar_docs

[Document(metadata={'source': ['love'], 'start_index': 0.0}, page_content="“People think a soul mate is your perfect fit, and that's what everyone wants. But a true soul mate is a mirror, the person who shows you everything that is holding you back, the person who brings you to your own attention so you can change your life. A true soul mate is probably the most important person you'll ever meet, because they tear down your walls and smack you awake. But to live with a soul mate forever? Nah. Too painful. Soul mates, they come into your life just to reveal another layer of yourself to you, and then leave. A soul mates purpose is to shake you up, tear apart your ego a little bit, show you your obstacles and addictions, break your heart open so new light can get in, make you so desperate and out of control that you have to transform your life, then introduce you to your spiritual master...”"),
 Document(metadata={'source': ['love', 'relationships', 'romance'], 'start_index': 0.0}, page_c

In [ ]:
similar_docs[0]

Document(metadata={'source': ['love'], 'start_index': 0.0}, page_content="“People think a soul mate is your perfect fit, and that's what everyone wants. But a true soul mate is a mirror, the person who shows you everything that is holding you back, the person who brings you to your own attention so you can change your life. A true soul mate is probably the most important person you'll ever meet, because they tear down your walls and smack you awake. But to live with a soul mate forever? Nah. Too painful. Soul mates, they come into your life just to reveal another layer of yourself to you, and then leave. A soul mates purpose is to shake you up, tear apart your ego a little bit, show you your obstacles and addictions, break your heart open so new light can get in, make you so desperate and out of control that you have to transform your life, then introduce you to your spiritual master...”")

In [ ]:
# below is the original entry from added dataset, just to cross check

{\n "quote": "\u201cPeople think a soul mate is your perfect fit, and that's what everyone wants. But a true soul mate is a mirror, the person who shows you everything that is holding you back, the person who brings you to your own attention so you can change your life. A true soul mate is probably the most important person you'll ever meet, because they tear down your walls and smack you awake. But to live with a soul mate forever? Nah. Too painful. Soul mates, they come into your life just to reveal another layer of yourself to you, and then leave. A soul mates purpose is to shake you up, tear apart your ego a little bit, show you your obstacles and addictions, break your heart open so new light can get in, make you so desperate and out of control that you have to transform your life, then introduce you to your spiritual master...\u201d",\n "author": "Elizabeth Gilbert,",\n "tags": "love"\n }

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature = 0)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
qa.invoke(query)

{'query': 'what does an ideal soulmate do?',
 'result': 'An ideal soulmate, according to the provided context, serves several profound purposes:\n\n1. **Acts as a Mirror**: They reflect back to you everything that is holding you back, bringing your attention to areas of your life that need change.\n2. **Facilitates Personal Growth**: They help you confront your ego, reveal your obstacles and addictions, and break your heart open so that new light can enter, prompting you to transform your life.\n3. **Provides Deep Connection**: They are someone with whom you can share your deepest thoughts, hopes, dreams, and disappointments without fear of judgment.\n4. **Offers Emotional Support**: They share in your excitement during good times and are not embarrassed to cry with you during tough times. They build you up and show you the things that make you special and beautiful.\n5. **Creates a Safe Space**: There is no pressure, jealousy, or competition with them. Instead, there is a quiet calmne

In [ ]:
query = 'which two things are infinite?'
qa.invoke(query)

{'query': 'which two things are infinite?',
 'result': 'According to the context provided, the two things that are infinite are the universe and human stupidity. This is a quote often attributed to Albert Einstein, although there is some debate about its authenticity.'}